# Web of Science (WOS)

In [7]:
import pandas as pd
import re
import sys

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',200)

In [40]:
def fill_NaN(df):
    '''Fill NaN entries with proper empty values
     Type  : dtype: Fill with
     string: "0"  : ''
     float : "float64" 
    '''
    for key in df.columns:
        if df[key].dtype=='O':
            df[key]=df[key].str.strip()
            df[key]=df[key].fillna('')
        elif df[key].dtype=='float64':
            df[key]=df[key].fillna(0.0)
    return df
def read_excel_fill_NaN(*args, **kwargs):
    '''Fill NaN entries with proper empty values
     Type  : dtype: Fill with
     string: "0"  : ''
     float : "float64" 
    '''
    df=pd.read_excel(*args, **kwargs)
    df=fill_NaN(df)
    return df

def add_SJR_info(journal_df,sjr_df,\
                sjr_open_acces_df=pd.DataFrame(),column_journal='SO'):
    if sjr_open_acces_df.shape[0]:
        if not column_journal in journal_df:
            sys.exit('%s no in journal DataFrame: %s' %(column_journal,journal_df))
        journal_df['Open_Access']='No'
        for joa in np.intersect1d( journal_df[column_journal].str.lower().str.strip().unique(),\
                                   sjr_oa.Title.str.lower().str.strip().unique() ):
            moa=journal_df[ journal_df[column_journal].str.lower() == joa ]
            for j in moa.index:
                journal_df.loc[j,'Open_Access']='Yes'    
            
    #for i in sjr_df.index: #Search upon 
    #moa=journal_df[ journal_df[column_journal].str.lower() == sjr_df.Title.str.lower().ix[i] ]
            
        return journal_df
        


Design:
Creates new full data frame joining information from several databases. 
The initial data frame is in Web of Science format, with tge prefix `wos_` in the column names:
The new data is added as addtional columns in either an existing row identified by
* DOI
* Title and if necessary vol, first author surname, journal etc.
In the current implementation only DOI will be working.

In [1]:
import time

In [21]:
import numpy as np

In [44]:
start=time.time()
wosa=add_SJR_info(wos,sjr,sjr_oa,column_journal='SO')
time.time()-start

1.6572577953338623

In [45]:
wosa[wosa.Open_Access=='Yes'].shape


(1308, 56)

In [39]:
start=time.time()
journal_df=wos
column_journal='SO'
journal_df['Open_Access']='No'
sjr_open_acces_df=sjr_oa
for i in sjr_open_acces_df.index:
    moa=journal_df[ journal_df[column_journal].str.lower() == sjr_open_acces_df.Title.str.lower().ix[i] ]
    for j in moa.index:
        journal_df.loc[j,'Open_Access']='Yes'
time.time()-start

19.14590883255005

In [20]:
add_SJR_info('wos_full_with_CR.xlsx',sjr_open_acces_df=sjr_oa)

SystemExit: SO no in journal DataFrame: wos_full_with_CR.xlsx

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
i=0
wos['Open_Access']='No'
for i in oaj.index:
    moa=wos[ wos.SO.str.lower() == oaj.Title.str.lower().ix[i] ]
    for j in moa.index:
        wos.loc[j,'Open_Access']='Yes'

In [41]:
wos=read_excel_fill_NaN('wos_full_with_CR.xlsx')

In [42]:
sjr_oa=pd.read_excel('scimago/journals_scimago_open_access.xlsx')
sjr=pd.read_excel('scimago/journals_scimago_all.xlsx')

In [13]:
wos[:1]

,AB,AF,AR,AU,BP,CR,DE,DT,EM,EP,FU,FX,GA,ID,IS,JI,LA,NR,PA,PD,PG,PI,PT,PU,PY,RP,SC,SN,SO,TC,TI,UT,VL,WC,DI,PM,RI,EI,OI,CL,CT,CY,SP,SU,BE,BN,HO,PN,SE,MA,SI,GP,CA,BA,BF
0,"Objectives: This work is intended to establish the prevalence of reverse smokers at the villages of Hato Nuevo, San Francisco and Cayo de Palma, Department of Sucre, Colombia, characterizing their...","Alvarez Gomez, Gloria J.\nAlvarez Martinez, Efrain\nJimenez Gomez, Raul\nMosquera Silva, Yolanda\nGaviria Nunez, Angela Maria\nGarces Agudelo, Adriana\nAlonso Duque, Alexander\nZabala Castano, Ale...",1111111172,"Gomez, GJA\nMartinez, EA\nGomez, RJ\nSilva, YM\nNunez, AMG\nAgudelo, AG\nDuque, AA\nCastano, AZ\nGonzalez, EE\nMillan, MI\nOssa, DR",E1,"Axell T, 1996, J ORAL PATHOL MED, V25, P49, DOI 10.1111/j.1600-0714.1996.tb00191.x\nBARIC JM, 1982, ORAL SURG ORAL MED O, V54, P424, DOI 10.1016/0030-4220(82)90389-9\nChang YC, 2001, J ORAL PATHOL...",oral cancer; oral premalignant lesions; reverse smoker; tobacco,Article,gloria@alvarez.nu,E8,CODI,"To the habitants of Hato Nuevo, San Francisco and Cayo de Palma, Sucre,\nColombia; to the field advisors for their help and attention in the\naccomplishment of this study and to the CODI by the pr...",352NW,SUBMUCOUS FIBROSIS; ANDHRA-PRADESH; SMOKING; LESIONS; INDIA;\nLEUKOPLAKIA; EPITHELIUM; CARCINOMA; CIGARETTE; NICOTINE,1,Med. Oral Patol. Oral Cir. Bucal,English,28,"CALLE DANIEL BALACIART N 4 PTA 17, VALENCIA, 46020, SPAIN",JAN,8,VALENCIA,J,MEDICINA ORAL S L,2008,"Gomez, GJA (reprint author), Univ Antioquia, Fac Dent, Calle 64 52-59, Medellin, Colombia.","Dentistry, Oral Surgery & Medicine",1698-4447,MEDICINA ORAL PATOLOGIA ORAL Y CIRUGIA BUCAL,4,"Reverse smokers's and changes in oral mucosa. Department of Sucre,\nColombia",WOS:000259504900001,13,"Dentistry, Oral Surgery & Medicine",,0,,,,,,,,,,,,,,,,,,,


In [18]:
'SO' in wos.columns.values

True

In [19]:
add_SJR_info(wos,sjr)

1


In [30]:
i=647
oaj.Title.str.lower().ix[i]

u'advances in high energy physics'

In [31]:
wos['Open_Access']='No'

In [32]:
i=0
wos['Open_Access']='No'
for i in oaj.index:
    moa=wos[ wos.SO.str.lower() == oaj.Title.str.lower().ix[i] ]
    for j in moa.index:
        wos.loc[j,'Open_Access']='Yes'

In [9]:
wos[wos.Open_Access=='Yes'].shape

(1308, 56)

#  Scopus

In [2]:
sc=pd.read_excel('scopus_all.xlsx')

In [16]:
sc=sc.rename_axis({'Source title':'Source_title'},1)
for key in sc.columns:
    if sc[key].dtype=='O':
        sc[key]=sc[key].str.strip()
        sc[key]=sc[key].fillna('')
    elif sc[key].dtype=='float64':
        sc[key]=sc[key].fillna(0.0)

In [17]:
sc

,﻿Authors,Title,Year,Source_title,Volume,Issue,Art. No.,Page start,Page end,Page count,Cited by,DOI,Link,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,Molecular Sequence Numbers,Chemicals/CAS,Tradenames,Manufacturers,Funding Details,Correspondence Address,Editors,Sponsors,Publisher,Conference name,Conference date,Conference location,Conference code,ISSN,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Source,EID
0,"González D.G., Solís M.P., Cantero M.T.R., Moncada M.D.R.O., Giraldo Á.F., Stein A., Díaz C.Á.-D.",The challenge to public health of the Millenium Development Goals: An approach from political epidemiology [El reto para la salud pública de los Objetivos de Desarrollo del Milenio: Un enfoque des...,2006.0,Gaceta Sanitaria,20,SUPPL. 3,,61,65,0.0,5.0,10.1157/13101091,https://www.scopus.com/inward/record.uri?eid=2-s2.0-34547700023&partnerID=40&md5=2155e0c6cdb235efd8a1953c4b49c370,"Área de Medicina Preventiva Y Salud Pública, Universidad de Alicante, Alicante, Spain; Facultad de Medicina, Universidad Autónoma de Yucatán, Yucatán, Mexico; Facultad Nacional de Salud Pública, U...","González, D.G., Área de Medicina Preventiva Y Salud Pública, Universidad de Alicante, Alicante, Spain, Observatorio de Políticas Públicas Y Salud (OPPS), Spain, Área de Medicina Preventiva Y Salud...","The Millennium Development Goals, and within these the Millennium Targets, constitute a working plan that strives to achieve basic goals within the field of health, eradicate poverty, and ensure c...",Development; Millenium Goals; Political epidemiology; WHO,epidemiology; motivation; politics; public health; review; social change; Epidemiology; Goals; Politics; Public Health; Social Change,,,,,,"González, D.G.; Área de Medicina Preventiva Y Salud Pública, Universidad de AlicanteSpain; email: Diana.Gil@ua.es",,,,,,,0.0,02139111,,,17433202.0,Spanish,Gaceta Sanit.,Review,Scopus,2-s2.0-34547700023
1,"Salazar E.J.G., Oscar Ortega L.",Sistema de búsqueda personalizada y recomendación de documentación científica,2006.0,Inteligencia Artificial,10,30,,25,42,0.0,0.0,,https://www.scopus.com/inward/record.uri?eid=2-s2.0-33845577371&partnerID=40&md5=7e9de49b24c81e399c97c8c8d1845688,"Universidad de Antioquia, Medellin, Colombia","Salazar, E.J.G., Universidad de Antioquia, Medellin, Colombia; Oscar Ortega, L., Universidad de Antioquia, Medellin, Colombia",[No abstract available],,,,,,,,"Salazar, E.J.G.; Universidad de Antioquia, Medellin, Colombia; email: esalazar@psl.com.co",,,,,,,0.0,11373601,,,0.0,Spanish,Inteligencia Artif.,Article,Scopus,2-s2.0-33845577371
2,"Colorado H.A., Vélez J., Salva H.R., Ghilarducci A.A.",Damping behavior of physical vapor-deposited TiN coatings on AISI 304 stainless steel and adhesion determinations,2006.0,Materials Science and Engineering A,442,1-2 SPEC. ISS.,,514,518,0.0,2.0,10.1016/j.msea.2006.04.133,https://www.scopus.com/inward/record.uri?eid=2-s2.0-33751081013&partnerID=40&md5=1e68fa50b8bf286de99eb71a1707d2ae,"Universidad de Antioquia, Mechanical Engineering Department, Calle 67 #53-108, Medellín, Colombia; Universidad Nacional de Colombia, School of Materials Engineering, Crra 80 #65-223, Bloque M1-410...","Colorado, H.A., Universidad de Antioquia, Mechanical Engineering Department, Calle 67 #53-108, Medellín, Colombia, Universidad Nacional de Colombia, School of Materials Engineering, Crra 80 #65-22...","Internal friction and elastic modulus measurements were carried out on AISI 304 stainless steel coated by physical vapor-deposited TiN, using a torsion pendulum and a vibrant-reed instrument. Some...",Adhesion; Internal friction; Stainless steel; Thin film coating,Adhesion; Damping; Elastic moduli; Enthalpy; Inorganic coatings; Internal friction; Martensitic transformations; Scanning electron microscopy; Stainless steel; Titanium nitride; X ray diffraction ...,,,,,,"Colorado, H.A.; Universidad de Antioquia, Mechanical Engineering Department, Calle 67 

In [20]:
i=0
sc['Open_Access']='No'
for i in oaj.index:
    moa=sc[ sc.Source_title.str.lower() == oaj.Title.str.lower().ix[i] ]
    for j in moa.index:
        sc.loc[j,'Open_Access']='Yes'
        

In [21]:
sc.shape

(8524, 41)

In [22]:
sc[sc.Open_Access=='Yes'].shape

(3026, 41)

In [23]:
kk=pd.read_excel('wos_open_access.xlsx')

In [24]:
kk.shape

(1075, 53)

In [33]:
wos.shape

(6644, 56)

Recheck journals

In [34]:
scoa=sc[sc.Open_Access=='Yes']

In [41]:
scoa.Source_title.unique()

array([u'Gaceta Sanitaria', u'Inteligencia Artificial',
       u'Informacion Tecnologica',
       u'Revista Latinoamericana de Psicologia', u'Iatreia',
       u'Revista Chilena de Nutricion', u'Historia Critica',
       u'Livestock Research for Rural Development',
       u'American Journal of Hematology',
       u'Revista Panamericana de Salud Publica/Pan American Journal of Public Health',
       u'Revista Medica de Chile', u'Salud Publica de Mexico',
       u'Cadernos de Saude Publica', u'Retrovirology',
       u'Journal of Infectious Diseases', u'Colombia Medica',
       u'Brazilian Journal of Medical and Biological Research',
       u'Memorias do Instituto Oswaldo Cruz', u'Index de Enfermeria',
       u'Revista Brasileira de Epidemiologia',
       u'Revista Chilena de Neuro-Psiquiatria',
       u'Revista Chilena de Pediatria', u'Malaria Journal',
       u'Revista de Saude Publica', u'Revista Colombiana de Entomologia',
       u'Revista Mexicana de Biodiversidad', u'Immunology', u'T

In [40]:
sc.Source_title.unique().shape

(2384,)